<a href="https://colab.research.google.com/github/Alexrosulek/Cs50/blob/main/gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
!nohup ollama serve &
!pip install pyOpenSSL==24.2.1


# Pull Ollama Models

!ollama pull qwen3:0.6b

!ollama pull gemma3:1b
!ollama pull gemma3:4b
# Install Packages
!pip install -q ollama crawl4ai aiohttp pillow beautifulsoup4 wikipedia googlesearch-python playwright nest_asyncio
!playwright install chromium
!nohup ollama serve &



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  24680      0 --:--:-- --:--:-- --:--:-- 24685
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InR

In [59]:
!nohup ollama serve &
import random
import subprocess
import wikipedia
import requests
from crawl4ai.content_filter_strategy import PruningContentFilter

from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from googlesearch import search
from crawl4ai import LLMConfig, LLMExtractionStrategy, CrawlerRunConfig,CacheMode
from crawl4ai.deep_crawling.filters import FilterChain, ContentRelevanceFilter
from crawl4ai.deep_crawling import BFSDeepCrawlStrategy
import json

from playwright.async_api import async_playwright

from crawl4ai.extraction_strategy import CosineStrategy

from crawl4ai.async_configs import BrowserConfig

from crawl4ai import AsyncWebCrawler,GeolocationConfig

import requests
from urllib.parse import urlparse

import asyncio
import time
import requests
import nest_asyncio
import httpx

API_BASE = "https://www.nearestdoor.com"  # Replace with actual server URL
CLIENT_ID = "client001"
HEARTBEAT_INTERVAL = 60  # seconds
SHOP_FLOW_STATIC = [
    "search", "aggregate", "createplan", "create",
    "find_available_fields", "extract_fields_from_aggregate", "fillintheshop"
]
# --------------------------------------------------------------------------- #
# 🧠  LIGHT‑WEIGHT LOCAL LLM EXECUTION                                       #
# --------------------------------------------------------------------------- #
class OllamaRunner:
    """
    `ollama run …`

    """
    def __init__(self, default_model: str = "gemma3:1b", default_timeout: int = 600):
        self.default_model = default_model
        self.default_timeout = default_timeout

    def run(self, prompt: str, model: str | None = None, timeout: int | None = None) -> str:
        model = model or self.default_model
        timeout = timeout or self.default_timeout
        print(f"🧠 Running Ollama: {model}")

        try:
            proc = subprocess.run(
                ["ollama", "run", model],
                input=prompt.encode("utf-8"),
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout,
            )

            raw_output = proc.stdout.decode("utf-8").strip()
            return re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL).strip()
        except Exception as e:
            print(f"❌ Ollama execution failed: {e}")
            return ""


# --------------------------------------------------------------------------- #
# 🌐  LOOK‑UP ENGINE                                                          #
# --------------------------------------------------------------------------- #
class LookupEngine:
    """
    – Validates every URL first
    – Google results exclude Yelp & Reddit and are content‑checked
    – Yelp & Reddit results are *also* content‑checked before ‘battling’
    – At most one Yelp URL & one Reddit URL are returned
    – Wikipedia returns at most one page (auto_suggest)
    """
    def __init__(self,  ollama_runner: OllamaRunner | None = None):

        self.llm_config = LLMConfig(provider="ollama/gemma3:1b")
        self.ollama = ollama_runner or OllamaRunner()
        self.crawler_manager = self.CrawlerManager()
    async def initialize(self):
        await self.crawler_manager.start()


    # ---------------------  LOW‑LEVEL HELPERS  ----------------------------- #
    class CrawlerManager:
        def __init__(self):
            self.crawler = None
        async def start(self):
            if self.crawler is None:
                self.crawler = AsyncWebCrawler(config=BrowserConfig())
                await self.crawler.__aenter__()

        async def stop(self):
            if self.crawler:
                await self.crawler.__aexit__(None, None, None)
                self.crawler = None


        async def crawl(self, url: str, config: CrawlerRunConfig | None = None):
            if not self.crawler:
                raise RuntimeError("Crawler not started")

            try:
                result = await self.crawler.arun(url=url, config=config or CrawlerRunConfig())
                return result
            except Exception as e:
                print(f"❌ crawl error for {url}: {e}")
                return None


        async def _normalize_and_validate_url(self, url: str) -> str | None:
          try:
              url = url.lower()
              if not url.startswith(("http://", "https://")):
                  url = "https://" + url

              parsed = urlparse(url)
              if (
                  parsed.scheme not in ["http", "https"]
                  or not parsed.netloc
                  or "." not in parsed.netloc
                  or " " in parsed.netloc
                  or "/http" in parsed.netloc
              ):
                  return None  # Invalid URL

              # ✅ Check if the URL is reachable using httpx (this was incorrectly indented before)
              async with httpx.AsyncClient(timeout=5) as client:  # Timeout should be in seconds, not 800ms
                  try:
                      response = await client.head(url, follow_redirects=True)
                      if response.status_code < 400:
                          return url
                      # Some servers reject HEAD, fallback to GET
                      response = await client.get(url, follow_redirects=True)
                      if response.status_code < 400:
                          return url
                      return False  # URL unreachable
                  except httpx.RequestError as e:
                      print(f"❌ HTTP check failed: {url} ({e})")
                      return None

          except Exception as e:
              print(f"❌ URL normalization failed: {e}")
              return None


    async def _extract_snippet(self, url, max_chars, min_chars):
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch(headless=True)
                context = await browser.new_context(
                    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
                    locale="en-US",
                    java_script_enabled=False,
                    permissions=["geolocation"],

    viewport={"width": 1280, "height": 720}
                )
                page = await context.new_page()

                print(f"Fetching: {url}")
                await page.goto(url, wait_until="domcontentloaded", timeout=30000)

                await page.wait_for_timeout(random.randint(1000, 3000))  # Randomized delay


                title = await page.title()
                desc = None

                # Try standard meta description first
                try:
                    desc = await page.locator('meta[name="description"]').get_attribute('content')
                except Exception:
                    # Try OpenGraph and Twitter metadata as fallback
                    for meta_tag in ["meta[property='og:description']", "meta[name='twitter:description']"]:
                        try:
                            desc = await page.locator(meta_tag).get_attribute('content')
                            if desc:
                                break
                        except Exception:
                            continue
                    if not desc:
                        print(f"No meta description found for {url}")

                heads = await page.locator("h1, h2, h3, h4, h5, h6").all_inner_texts()
                para_elements = await page.locator("p").all_inner_texts()

                parts = []
                if title:
                  parts.append("Title: " + title.strip())
                if desc:
                  parts.append("Desc: " + desc.strip())
                if heads:
                  parts.append("Headers: ")
                  parts.extend([ " , "+ h.strip() for h in heads[:6] if h.strip()])

                # Intelligent Paragraph Extraction
                para_candidates = para_elements[0:20]

                # Force convert to string and strip, without any type checking
                paras = [str(p).strip() for p in para_candidates if len(str(p).strip()) > 30]



                try:
                    aria_labels = await page.locator("[aria-label]").evaluate_all("els => els.map(e => e.getAttribute('aria-label'))")
                    alt_texts = await page.locator("[alt]").evaluate_all("els => els.map(e => e.getAttribute('alt'))")
                    contextual_info = [text.strip() for text in (aria_labels + alt_texts) if text and len(text.strip()) > 30]
                    paras.extend(contextual_info[:4])  # Add up to 2 contextual info pieces
                except Exception:
                    pass
                if paras:
                  parts.append("Paragraphs: ")
                  for para in paras:  # Limit to top 4 paragraphs/info pieces
                      parts.append(" , "+para)

                await browser.close()

                result = "\n".join(parts)


                print("ddj")
                if len(str(result)) < min_chars:
                    print(f"❌ Extracted content too short: {len(result)} chars")
                    return None
                result = result[:max_chars]
                return f"Snippet From {url}:\n{result[:max_chars]}"

        except Exception as e:
            print(f"❌ Playwright extraction failed for {url}: {e}")
            return None




    async def _basic_url_checker(self, url: str, shop_name: str, shop_type: str, state: str, city:str) -> bool:
        try:

            """
            Improved relevance check for a URL:
            1. Uses Crawl4AI's ContentRelevanceFilter for semantic similarity.
            2. Falls back to LLM prompt decision if semantic check is inconclusive.
            """

            if not url:
                return False




            # Step 2: Fallback — Quick Snippet and LLM Yes/No Decision
            snippet = await self._extract_snippet(url, 750, 80)
            if not snippet:
                print("no snippet")
                return False

            prompt = (
                f"Is the website {url} related to '{shop_name}' or the place its located in, {state}, {city}? "
                f"Be very lient and allow all websites related to {shop_type}. If the name {shop_name} or {state}, {city} is present or related subject like info on this place then it is likely relavent. Answer only `true` or `false` or `none`.\n\n{snippet}"
            )

            decision = self.ollama.run(prompt, 'qwen3:0.6b')
            print(decision)
            print(prompt)
            if "true" in decision.lower():
                print(f"✅ LLM confirmed relevance for: {url}")
                return True

            print(f"⚠️ URL deemed irrelevant: {url}")
            return False

        except Exception as e:
            print(f"error in basicc checker", {url}, {shop_name}, {shop_type}, e)
            return False
    @staticmethod
    def get_semantic_query(shop_type, shop_name):
        queries = {
            "church": f"{shop_type}, {shop_name}, history, review, hours, muslim, phone, church, christian, church events, holiday schedules, mass times, sermons, church history, community programs, accessibility options, FAQs, donation methods, parking, contact information",
            "plasma_center": f"{shop_type}, {shop_name}, history, review, stocked brands, review, hours, phone, plasma, plasma donation requirements, compensation rates, donor reward, donor eligibility, contact details, operating hours, health guidelines, FAQ, appointment scheduling, safety procedures",
            "thrift_store": f"{shop_type}, {shop_name}, history, review, stocked brands, second hand,  review, hours, phone, thrift, store hours, donation guidelines, accepted items, discounts, sales events, store history, accessibility, contact info, volunteer programs, reviews",
            "dog_park": f"{shop_type}, {shop_name}, history, review, water, shade, agility equipment, park, review, hours, phone, dog, dog park hours, leash rules, pet-friendly areas, dog-friendly facilities, park amenities, accessibility options, entry fees, safety tips, events, pet policies, reviews",
        }
        return queries.get(shop_type.lower(), "business information, contact details, operating hours, reviews, FAQs, history")

    async def _get_site_content(

            self,
            url: str,
            shop_name: str,
            shop_type: str,
state:str, city:str
            ) -> str | None:
        try:
            print("scraping")
        # Step 1: Semantic Filter Based on Shop Type
            #semantic_query = self.get_semantic_query(shop_type, shop_name)

            llm_strategy = LLMExtractionStrategy(
            llm_config = LLMConfig(provider="ollama/qwen3:0.6b"),

          extraction_type="block",
          instruction=f"Extract all data and useful information, remove noise",
          chunk_token_threshold=650,
          overlap_rate=0.05,
          apply_chunking=True,
          input_format="fit_markdown",   # or "html", "fit_markdown"


      )
            prune_filter = PruningContentFilter(
                        threshold=0.2,
                        threshold_type="dynamic",  # or "dynamic"
                        min_word_threshold=3
                    )

            md_generator = DefaultMarkdownGenerator(content_filter=prune_filter)


            crawl_config = CrawlerRunConfig(
                cache_mode=CacheMode.BYPASS,
                extraction_strategy=llm_strategy,
                  excluded_tags=["head","style", "header", "footer", "script"],
        markdown_generator=md_generator,
                   magic=True,  # Simplifies a lot of interaction
            remove_overlay_elements=True,
              locale="en-US",  # US English
            timezone_id="America/New_York",  # Eastern Time (You can also use 'America/Los_Angeles' for Pacific Time)
          geolocation=GeolocationConfig(
            latitude=40.7128,
            longitude=-74.0060,
            accuracy=100  # Accuracy in meters (optional)
        ),


    page_timeout=60000,


            )

            result = await self.crawler_manager.crawl(url, config=crawl_config)
            print(result)
            if not result or not hasattr(result, '__iter__'):
              print(f"❌ No crawl results for {url}")
              return None

            successful_results = [r for r in result if getattr(r, 'success', False) and getattr(r, 'extracted_content', None)]

            if not successful_results:
                print(f"❌ content extraction failed for {url}")
                return None

            # Combine extracted content from successful results
            content_chunks = [item.extracted_content for item in successful_results]
            combined_content = "\n\n".join(content_chunks)
            return combined_content



        except Exception as e:
            print(f"❌ Failed to parse site content: {e}")
            return None
    # ---------------------  LOOK‑UP ROUTINES  ------------------------------ #
    async def wikipedia_lookup(self, name: str, city: str, shop_type: str) -> str | None:
        try:
            query = f"{name} {city} {shop_type}".strip()
            print(f"📚 Wikipedia lookup → {query}")
            page = wikipedia.page(query, auto_suggest=True)
            content = page.content

            if len(content) <= 2000:
                return content
            chunks = [content[i:i + 500] for i in range(0, len(content), 500)]
            # Fallback-safe middle extraction
            if len(chunks) > 6:
                middle = chunks[2:-2]  # Remove first and last 2 chunks
                if not middle:
                    middle = chunks  # If middle is empty, fallback to all chunks
            else:
                middle = chunks

            # Intelligent selection
            if len(middle) > 6:
                selected = random.sample(middle, 6)  # Randomly select 6 if too many
            else:
                selected = middle  # Take all available if 6 or fewer
            formatted_chunks = [f"\nWIKI CHUNK {idx + 1}:\n{chunk}" for idx, chunk in enumerate(selected)]

            return f"ALL EXTRACTED WIKIPEDIA SEARCH INFO FOR {name}:\n" + "\n".join(formatted_chunks)

        except Exception as e:
            print(f"❌ Wikipedia fetch failed: {e}")
            return None

    async def search_lookup(self,  name: str,  shop_type: str, query: str, placename: str, amount: int,state:str, city:str, isyelp: bool) -> str | None:
        try:
            print(f"🔎 {placename} search → {query}")
            print("dd")
            raw = list(search(query, amount))
            if not isyelp:
                print("ddf")
                candidate_urls = [
                      u for u in raw
                      if all(excl not in str(u).lower() for excl in ["yelp", "reddit", "wiki", "nearestdoor", "facebook", "twitter"])
                  ]
                print("bfd")
            else:
                print("rgrg")
                candidate_urls = [
                      u for u in raw
                      if ("yelp" in str(u).lower() or "reddit" in str(u).lower())
                      and all(excl not in str(u).lower() for excl in ["wiki", "nearestdoor", "facebook", "twitter"])
                  ]
                print("frg")

            good_content = []

            print("ddd")
            for i, url in enumerate(candidate_urls):
                print(url, "url canidate")
                urld = await self.crawler_manager._normalize_and_validate_url(url)
                if not urld:
                    continue

                if not await self._basic_url_checker(urld, name, shop_type, state, city):
                    continue

                content = await self._get_site_content(urld, name, shop_type,state, city)
                if content:
                    good_content.append(f"\n← {placename} SEARCH DATA SITE {i} FROM: {url}\n {content}")

            return f"ALL EXTRACTED {placename} SEARCH DATA FOR {name}, {city}\n".join(good_content) if good_content else None
        except Exception as e:
            print(f"❌ Failed to parse {placename} search lookup results: {e}")
            return None
    # ---------------------  PUBLIC ENTRY POINT  ---------------------------- #
    async def combined_search(self, name: str, city: str, state: str, shop_type: str, website_url: str) -> tuple[bool, str | None, None]:
        print("🌐 Starting combined search…")

        Google_query = f"{name} {city} {state} {shop_type} "

        if website_url:
            Official_query = f"{name} site: {website_url} "

        g_res = await self.search_lookup(name, shop_type, Google_query, "Google", 10,state, city,  False)
        w_res = await self.wikipedia_lookup(name, city, shop_type)
        o_res = None
        if website_url:
            o_res = await self.search_lookup(name, shop_type, Official_query, f"Official Website of {name}", 5,state, city, True)
        main = ""


        if g_res:
            main += g_res
        if w_res:
            main += w_res
        if o_res:
            main += o_res
        print('sss')
        if len(main) < 500:
            print("❌ Not enough content gathered.")
            return False, None, None

        print("✅ Combined search complete.")
        return True, main, None
# --------------------------------------------------------------------------- #
# 📦  HIGH‑LEVEL CONTENT SUMMARIZER                                          #
# --------------------------------------------------------------------------- #
class ContentSummarizer:
    """
    Reduce a large blob of text about a specific business down to ≤ max_final_chars
    while preserving high‑value facts. Uses multi-stage LLM summarisation with
    chunk filtering and escalation if necessary.
    """
    def __init__(
        self,
        ollama_runner: OllamaRunner,
        shop_name: str,
        shop_type: str,
        city: str | None = None,
        state: str | None = None,
        max_final_chars: int = 6000,
        min_final_chars: int = 500,
    ):
        self.ollama = ollama_runner
        self.shop_name = shop_name
        self.shop_type = shop_type
        self.city = city or ""
        self.state = state or ""
        self.max_final_chars = max_final_chars
        self.min_final_chars = min_final_chars

    # ----------------- Internal Helpers ----------------- #
    def _clean_raw_content(self, content: str) -> str:
        lines = content.splitlines()
        cleaned, seen = [], set()
        NOISE = [
            "cookie policy", "all rights reserved", "subscribe", "advertisement",
            "accept cookies", "privacy policy", "terms of service", "sign in", "cookie"
        ]
        for line in lines:
            line = line.strip()
            lo = line.lower()
            if len(line) < 30 or lo in seen:
                continue
            if any(noise in lo for noise in NOISE):
                continue
            seen.add(lo)
            cleaned.append(line)
        return "\n".join(cleaned)

    def _chunk_text(self, text: str, chunk_size: int) -> list[str]:
        return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    def _filter_chunks(self, chunks: list[str], model: str = "gemma3:1b") -> list[str]:
        try:
            good = []
            for chunk in chunks:
                prompt = (
                    f"Is the following content useful for creating a profile for the {self.shop_type} "
                    f"'{self.shop_name}'? Only reply 'true' or 'false'.\n\n{chunk}"
                )
                decision = self.ollama.run(prompt, model=model).strip().lower()
                if "true" in decision:
                    good.append(chunk)

            return good
        except Exception as e:
            print(f"❌ Failed to filter chunk: {e}")
            return None

    def _build_prompt(self, text_chunk: str) -> str:
        return (
            f"You are creating a SHORT high-quality summary for "
            f"{self.shop_type} **{self.shop_name}** "
            f"{'in ' + self.city if self.city else ''}{', ' + self.state if self.state else ''}.\n\n"
            f"Keep ALL USEFUL DATA"
            "KEEP SERVICES.\n"
            f"KEEP URLS ONLY WHEN IT IS THE LITERAL {self.shop_name}'s WEBSITE.\n"
            f"KEEP ALL GOOD INFO AND HISTORY, FACTS, INFO, Extract all usefull info, Example: \n"
            f"• Keep Contact info (phone, email)\n"
            f"• Keep Official website URLs only if directly related to {self.shop_name}\n"
            f"• Keep Operating hours, holiday hours, KEEP ALL USEFUL ANY DATA\n"
            f"• Keep Brands Price range, services, FAQs, key facts, reviews, history\n"
            f"• Keep Reviews sentiment, pros/cons, unique selling points\n"
            f"• Keep Awards, certifications, reviews, press articles\n\n"
            f" remove unrelated info.\n"

            f"--- SOURCE TEXT START ---\n{text_chunk}\n--- SOURCE TEXT END ---"

        )

    def _summarize_with_ollama(self, text: str, model: str) -> str:
        try:

            prompt = self._build_prompt(text)
            return self.ollama.run(prompt, model=model)
        except Exception as e:
            print(f"❌ Failed to summarize w ollama: {e}")
            return None
    def summarize_chunks(
        self,
        content: str,
        chunk_size: int = 2000,
        initial_model: str = "gemma3:1b",

    ) -> str:
        try:
            chunks = self._chunk_text(content, chunk_size)
            content = self._filter_chunks(chunks)
            if not content:
                content = chunks
            filtered_content = ''.join(content)
            if len(filtered_content) < self.min_final_chars:
                return None
            if len(filtered_content) < self.max_final_chars:
                return filtered_content

            summarized_chunks = []

            for idx, chunk in enumerate(chunks, start=1):
                print(f"📚 Summarizing chunk {idx}/{len(chunks)}...")
                summary = self._summarize_with_ollama(chunk, model=initial_model)

                if not summary or len(summary) < 50:
                    print(f"⚠️ Failed to summarize chunk {idx}, keeping raw content.")
                    summary = chunk  # Fallback to raw content if summary failed

                summarized_chunks.append(f"### CHUNK {idx} SUMMARY:\n{summary}")

                # Early exit: check if adding all remaining raw chunks without summarizing fits within limit
                combined_so_far = "\n\n".join(summarized_chunks)
                remaining_raw = "".join(chunks[idx:])  # Remaining chunks after current one

                if len(combined_so_far) + len(remaining_raw) <= self.max_final_chars:
                    print(f"✅ Early exit: current summary + remaining raw fits within limit. Skipping further summarization.")
                    for r_idx, remaining_chunk in enumerate(chunks[idx:], start=idx + 1):
                        summarized_chunks.append(f"### CHUNK {r_idx} (Raw):\n{remaining_chunk}")
                    break

            combined_summary = "\n\n".join(summarized_chunks)
            if len(combined_summary) < self.min_final_chars:
                return None
            # Final trim if absolutely necessary
            if len(combined_summary) > self.max_final_chars:
                print("⚠️ Final combined summary exceeds character limit. Trimming result.")
                return combined_summary[:self.max_final_chars]

            return combined_summary
        except Exception as e:
            print(f"❌ Failed to sumarrize chunks: {e}")
            return None


    def summarize_content(self, raw_content: str) -> str:
        print("🧹 Cleaning raw content...")
        cleaned = self._clean_raw_content(raw_content)
        if len(cleaned) <= self.max_final_chars:
            print("✅ Cleaned content fits within final character limit.")
            return cleaned

        final_summary = self.summarize_chunks(
        content=cleaned,           # The large raw text content you want to reduce
        chunk_size=1500,                # Size of each chunk before summarizing
        initial_model="gemma3:1b",      # Start with the lightweight model

    )
        if final_summary is None:
            print("❌ Summarization failed. Returning cleaned content instead.")
            return cleaned[:self.max_final_chars]

        print("🎉 Final summarization complete.")
        return final_summary

import re
import json

class Smartypants:
    def __init__(self, ollama_runner: OllamaRunner):
        self.ollama = ollama_runner

    def _run4(self, prompt: str) -> str:
        return self.ollama.run(prompt, model="gemma3:4b",)
    def _run1(self, prompt: str) -> str:
        return self.ollama.run(prompt, model="gemma3:1b")

    # ------------ PLAN ------------ #
    def create_plan(self, aggregate: str,shop_name: str, shop_type: str,  city: str, state: str) -> tuple[bool, list[str]]:
        prompt = (
            "You have an aggregated summary about a "

            f"place called {shop_name} {shop_type} in {city}, {state}\n\n"
            f"{aggregate}\n\n"
            "Which content sections can confidently be generated based on this?\n"
            "Options: article, faq, history.\n"
            "Reply with a correct python comma-separated list of available sections to write about, nothing else, Options: article, faq, history."
        )
        try:
            count = 0
            resp = self._run4(prompt).lower()
            valid = {"article", "faq", "history"}
            print(resp)
            for s in valid:
                if s not in resp:
                    count +=1
            if count == 3:
                return False, []

            return True, [s for s in valid if s in resp]
        except Exception as e:
            print(f"❌ create_plan error: {e}")
            return False, []

    def check_aggregate_quality(self, shop_name: str, aggregate: str,shop_type: str,  city: str, state: str) -> bool:
        prompt = (
            f"Check if this content contains usefull info about {shop_name} {shop_type} in {city}, {state}.\n\n{aggregate}\n\n"
            "Reply only `true` or `false`."
        )
        try:
            return "true" in self._run1(prompt).lower()
        except Exception as e:
            print(f"❌ quality check error: {e}")
            return False

    # ------------ SECTION VALIDATION / FIX ------------ #
    def validate_section_html(self, shop_name: str, section: str, text: str) -> bool:
        prompt = (
            f"Validate the following text for section '{section}'. It is supposed to be about '{shop_name}'.\n\n{text}\n\n"
            "Rules:\n- No HTML.\n- No irrelevant info. Is it useful and no format or weird characters? Nothing Else, Nothing before or after our content\n"
            "- Only factual, structured, and clear content.\n- Reply `true` or `false` only."
        )
        return "true" in self._run1(prompt).lower()

    def fix_section_html(self, shop_name: str, section: str, text: str) -> str | None:
        prompt = (
            f"Clean and fix this section '{section}'s format. It is about '{shop_name}'.\n\n{text}\n\n"
            "Rules:\n- No HTML, Is it useful and and no format or weird characters? asterisks, or irrelevant info.\n"
            "Return only the final cleaned text for consumers on nearestdoor.com to read, no junk, no explanations, nothing else, nothing before or after our content. Only return the corrected text."
        )
        return self._run4(prompt).strip()

    # ------------ JSON & FIELD EXTRACTION ------------ #
    def extract_clean_json_structure(self, text: str) -> dict | None:
        try:
            match = re.search(r"\{.*?\}", text.strip(), re.DOTALL)
            if not match:
                return None

            match_text = match.group(0).lower()
            exclusion_keywords = ['n/a', 'n-a', 'none', 'false', 'na', 'cant', 'not', 'found', 'unable', '{{', '()', 'unavailable']

            if any(bad in match_text for bad in exclusion_keywords):
                return None

            json_ready = match.group(0).replace("'", '"')
            return json.loads(json_ready)

        except Exception:
            return None


    def extract_available_fields(self, aggregate: str,shop_name: str, shop_type: str,  city: str, state: str) -> tuple[bool, list[str]]:
        try:
            field_list = list(FIELD_EXTRACTORS.keys())
            field_str = ', '.join(field_list)
            prompt = (
                f"Analyze the content:\n\n{aggregate}\n\n"
                f"Whos data we want {shop_type}, {shop_name}. "
                f"Which of these fields can be confidently extracted?\n{field_str}\n"
                "Reply ONLY with the field keys that are present in the text, as ONLY the correct format requested. no junk. Nothing else. "
                "If none, reply exactly 'FALSE'."
            )
            response = self._run4(prompt).strip().lower()
            if "false" in response:
                return True, []
            detected = [field for field in field_list if field.lower() in response]
            return True, detected
        except Exception as e:
            print(f"❌ extract_available_fields error: {e}")
            return False, []
    def validate_extracted_field_value(self, field: str, value) -> bool:
        """
        Validate the extracted field value using LLM and manual schema checks.

        - If it's a JSON list, remove invalid entries.
        - If it's invalid after cleaning, return False.
        """
        # LLM-Based Validation Prompt
        prompt = (
            f"Validate this extracted value for field '{field}':\n\n{value}\n\n"
            "Is this a valid and correct value and format for the specified field requested? Is it weird for the field or contain none values? Reply ONLY `true` or `false`."
        )
        llm_decision = "true" in self._run1(prompt).lower()

        # If LLM says it's invalid, fail immediately
        if not llm_decision:
            print(f"❌ LLM validation failed for field '{field}'.")
            return False



        # Final check for singular values
        return True

    def extract_fields(
        self, aggregate: str, available_fields: list[str],
        shop_name: str, shop_type: str, city: str, state: str
    ) -> tuple[bool, dict]:
        extracted = {}
        try:
            for field in available_fields:
                try:
                    prompt = (
                        f"{FIELD_EXTRACTORS[field]}\n\nContent:\n{aggregate}\n\n"
                        f"Return ONLY the valid structure requested. Respond with nothing but the correct format requested. "
                        f"If none, say 'none'. Nothing else, nothing before or after our content. Data about {shop_type}, {shop_name}."
                    )
                    raw_value = self._run4(prompt).strip()
                    final_value = self.extract_clean_json_structure(raw_value) or raw_value

                    if self.validate_extracted_field_value(field, final_value):

                        extracted[field] = final_value

                except Exception as inner_e:
                    print(f"⚠️ Field extraction failed for '{field}': {inner_e}")
                    continue

            return True, extracted if extracted else {}

        except Exception as e:
            print(f"⚠️ extract_fields failed: {e}")
            return False, {}


    # ------------ SECTION GENERATION ------------ #
    def create_sections(
        self, shop_name: str, shop_type: str, aggregate: str,
        approved_sections: list[str], city: str | None = None, state: str | None = None
    ) -> tuple[bool, dict]:
        def _generate(section: str, prompt: str) -> str | None:
            raw = self._run4(prompt).strip()
            if self.validate_section_html(shop_name, section, raw):
                return raw
            fixed = self.fix_section_html(shop_name, section, raw)
            return fixed if fixed and self.validate_section_html(shop_name, section, fixed) else None

        location = f"in {city}, {state}" if city or state else ""
        base_instr = (
            f"You are writing for nearestdoor.com about our listings, about the {shop_type} '{shop_name}' {location}. "
            "You will get a summary and write useful information according to your assignment which consumers will read on the nearestdoor.com website about this place, no bad info or bad formatting"
            "Be factual, SEO-friendly, help the users learn use this place and learn about it. no unrelated info, no HTML or asterisks. nothing else, nothing before or after our content. DO NOT USE * "
        )

        sections = {}
        try:
            if "article" in approved_sections:
                prompt = f"{base_instr}\n\nContent:\n{aggregate}\n\nAssignment: Write a detailed article. Write an article about {shop_name} for nearestdoor.com."
                result = _generate("article", prompt)
                if result:
                    sections["article"] = result

            if "faq" in approved_sections:
                prompt = f"{base_instr}\n\nContent:\n{aggregate}\n\nAssignment: Write a detailed FAQ. Write an FAQ about {shop_name} for nearestdoor.com."
                result = _generate("faq", prompt)
                if result:
                    sections["faq"] = result

            if "history" in approved_sections:
                prompt = f"{base_instr}\n\nContent:\n{aggregate}\n\nAssignment: Write the history section about {shop_name} for nearestdoor.com."
                result = _generate("history", prompt)
                if result:
                    sections["history"] = result

            return True, sections
        except Exception as e:
            print(f"❌ create_sections error: {e}")
            return False, {}

    # ------------ FULL WORKFLOW ------------ #
    def process(
        self, shop_name: str, shop_type: str, aggregate: str,
        city: str | None = None, state: str | None = None
    ) -> dict:
        result = {"plan": [], "sections": {}, "fields": {}}

        if not self.check_aggregate_quality(shop_name, aggregate, shop_type, city, state):
            print("❌ Aggregate failed quality check.")
            return None

        ok, plan = self.create_plan(aggregate, shop_name, shop_type, city, state)
        if not ok or not plan:
            print("❌ No sections can be created.")
            return None
        result["plan"] = plan

        ok, sections = self.create_sections(shop_name, shop_type, aggregate, city, state)
        if ok:
            result["sections"] = sections

        ok, available = self.extract_available_fields(aggregate, shop_name, shop_type, city, state)
        if ok and available:
            ok, fields = self.extract_fields(aggregate, available,shop_name, shop_type,city, state)
            if ok:
                result["fields"] = fields

        return result

import re
import json

def is_non_empty_string(value) -> bool:
    return isinstance(value, str) and len(value.strip()) > 0

def is_valid_json(value) -> bool:
    try:
        json.loads(value)
        return True
    except Exception:
        return False

def is_valid_phone(value) -> bool:
    return bool(re.fullmatch(r"\d{3}-\d{3}-\d{4}", value.strip()))

def is_valid_email(value) -> bool:
    return bool(re.fullmatch(r"[^@\s]+@[^@\s]+\.[a-zA-Z0-9]+", value.strip()))

def is_valid_url(value) -> bool:
    return isinstance(value, str) and value.strip().startswith("http")

def is_valid_dict(value) -> bool:
    try:
        return isinstance(json.loads(value), dict)
    except Exception:
        return False

def is_valid_list(value) -> bool:
    try:
        return isinstance(json.loads(value), list)
    except Exception:
        return False

def is_positive_integer_or_string(value) -> bool:
    try:
        return int(str(value).strip()) > 0
    except Exception:
        return False

FIELD_VALIDATORS = {
    # Contact Info
    "extract_phone": is_valid_phone,
    "extract_email": is_valid_email,
    "extract_website": is_valid_url,

    # Structured Fields
    "extract_categories": is_valid_list,
    "extract_operating_hours": is_valid_dict,
    "extract_holiday_hours": is_valid_dict,
    "extract_delivery_services": is_valid_list,
    "extract_social_media": is_valid_dict,
    "extract_stocked_brands": is_valid_list,
    "extract_inventory_categories": is_valid_dict,
    "extract_customer_reviews": is_valid_list,

    # Event / Misc
    "extract_admission": is_non_empty_string,
    "extract_date_available": is_non_empty_string,
    "extract_attendance_amount": is_positive_integer_or_string,
    "extract_exhibitor_amount": is_positive_integer_or_string,
}
FIELD_EXTRACTORS = {
    # Contact Information
    "extract_phone": (
        "Extract the phone number in this format: 727-237-2132. "
        "Return ONLY the number, no quotes, no text, no comments, no markup."
    ),
    "extract_email": (
        "Extract the email address. Example: example@mail.com. "
        "Return ONLY the email address, no quotes, no text, no extras."
    ),
    "extract_website": (
        "Extract the official website URL. Example: https://website.com. "
        "Return ONLY the URL, no quotes, no text, no markup."
    ),

    # JSON / Structured Fields
    "extract_categories": (
        "Extract the product/service categories in JSON list format. "
        "Example: ['Thrift Store', 'Charity']. Return ONLY the JSON array."
    ),
    "extract_operating_hours": (
        "Extract weekly operating hours in JSON dictionary format. "
        "Example: {'monday': '9:00 AM - 5:00 PM', 'sunday': 'Closed'}. "
        "Return ONLY the JSON object."
    ),
    "extract_holiday_hours": (
        "Extract holiday-specific hours in JSON dictionary format. "
        "Example: {'2024-12-25': 'Closed', '2024-12-31': '10:00 AM - 4:00 PM'}. "
        "Return ONLY the JSON object."
    ),
    "extract_delivery_services": (
        "Extract available delivery services in JSON list format. "
        "Example: ['Uber Eats', 'Self Delivery']. Return ONLY the JSON array."
    ),
    "extract_social_media": (
        "Extract social media links in JSON dictionary format. "
        "Example: {'facebook': 'https://facebook.com/example', 'instagram': 'https://instagram.com/example'}. "
        "Return ONLY the JSON object."
    ),
    "extract_stocked_brands": (
        "Extract stocked brands in JSON list format. "
        "Example: ['Nike', 'Adidas']. Return ONLY the JSON array."
    ),
    "extract_inventory_categories": (
        "Extract inventory categories in JSON dictionary format. "
        "Example: {'Apparel': ['Shirts', 'Hoodies']}. Return ONLY the JSON object."
    ),
    "extract_customer_reviews": (
        "Extract customer reviews in JSON list format. "
        "Example: [{'user': 'John', 'comment': 'Great store!', 'rating': 5}]. "
        "Return ONLY the JSON array."
    ),

    # Event / Scheduling
    "extract_admission": (
        "Extract the admission cost or entry fee. Return ONLY the plain text, no prefixes or suffixes."
    ),
    "extract_date_available": (
        "Extract the available date range or date description. "
        "Example: 'Available from May 1st to June 30th'. Return ONLY the plain text."
    ),
    "extract_attendance_amount": (
        "Extract expected attendance as a number. Example: 500. Return ONLY the number or numeric string."
    ),
    "extract_exhibitor_amount": (
        "Extract expected number of exhibitors. Example: 12. Return ONLY the number or numeric string."
    ),
}

class NearestDoorClient:
    def __init__(self, smartypants, lookup_engine, ollama,  client_id=CLIENT_ID, api_base=API_BASE):
        self.client_id = client_id
        self.api_base = api_base
        self.ollama = ollama
        self.lookup_engine = lookup_engine

        self.last_heartbeat = 0


        self.smartypants = smartypants

    def _api_get(self, endpoint, params=None):
        try:
            print(f"📡 GET → {endpoint}")
            response = requests.get(f"{self.api_base}{endpoint}", params=params or {}, timeout=30)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"❌ GET failed: {e}")
            return None

    def _api_post(self, endpoint, data):
        try:
            print(f"📡 POST → {endpoint}")
            response = requests.post(f"{self.api_base}{endpoint}", json=data, timeout=30)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"❌ POST failed: {e}")
            return None

    def get_task(self):
        res = self._api_get("/next-task", params={"client_id": self.client_id})
        if res and res.status_code == 200:
            task = res.json()
            if isinstance(task, dict) and "task_id" in task:
                return task
            print(f"⚠️ Invalid task structure received: {task}")
        return None

    def send_heartbeat(self, current_task_id=None):
        data = {"client_id": self.client_id}
        if current_task_id:
            data["task_id"] = current_task_id
        self._api_post("/heartbeat", data)
        print("🫀 Heartbeat sent.")

    async def handle_task(self, task):
        task_id =task.get("task_id")
        task_type =task.get("task_type")
        if not task_id or not task_type:
            print("❌ Invalid task format.")
            return

        print(f"▶️ Handling task {task_type} (ID: {task_id})")

        result, summary, mainstring, images = False, None, None, None
        aggregateplan, createdinfo, extractedfields, foundfields = None, None, None, None
        print(task)
        name = task['target'].get("name")
        city = task['target'].get("city")
        state = task['target'].get("state")
        website_url = task['target'].get("website", None)

        shop_type = task['target'].get("shop_type")
        aggregate = task['target'].get("aggregate", "")
        plan = task['target'].get("plan", [])
        fields = task['target'].get("fields", [])

        match task_type:
            case "search":
                result, mainstring, images = await self.lookup_engine.combined_search(name, city, state, shop_type, website_url)
                result = str(result)
            case "aggregate":
                summarizer = ContentSummarizer(self.ollama, name, shop_type, city, state)
                summary = summarizer.summarize_content(aggregate)
                result = bool(summary)

            case "createplan":
                result, aggregateplan = self.smartypants.create_plan(aggregate, name, shop_type, city, state)

            case "create":
                result, createdinfo = self.smartypants.create_sections(name, shop_type, aggregate, plan, city, state)

            case "find_available_fields":
                result, foundfields = self.smartypants.extract_available_fields(aggregate, name, shop_type, city, state)

            case "extract_fields_from_aggregate":
                result, extractedfields = self.smartypants.extract_fields(aggregate, fields, name, shop_type, city, state)

            case _:
                print(f"❌ Unknown task type: {task_type}")
                return
        if result:
                print(f"📤 Submitting result for {task_type} ({task_id})")
                try:
                    if task_type == 'search':
                        res = self._api_post(f"/submit/{task_id}", {"status": "success", "mainstring": mainstring, "client_id": CLIENT_ID})

                    if task_type == 'aggregate':
                        if summary:
                            res = self._api_post(f"/submit/{task_id}", {"status": "success", "summary": summary, "client_id": CLIENT_ID})
                        else:
                            print("nosummary")
                            res = self._api_post(f"/submit/{task_id}", {"status": "fail", "client_id": CLIENT_ID})


                    if task_type == 'createplan':
                        res = self._api_post(f"/submit/{task_id}", {"status": "success", "aggregateplan": aggregateplan, "client_id": CLIENT_ID})
                    if task_type == 'create':
                        res = self._api_post(f"/submit/{task_id}", {"status": "success", "createdinfo":createdinfo, "client_id": CLIENT_ID})
                    if task_type == 'find_available_fields':
                        res = self._api_post(f"/submit/{task_id}", {"status": "success", "foundfields": foundfields, "client_id": CLIENT_ID})
                    if task_type == 'extract_fields_from_aggregate':
                        res = self._api_post(f"/submit/{task_id}", {"status": "success", "extractedfields": extractedfields, "client_id": CLIENT_ID})

                    print(f"Server responded: {res.status_code} - {res.text}")
                    if res:
                      if res.status_code == 200:
                          print(f"✅ Submitted: {task_type}")
                      else:
                          print(f"❌ Submit failed: {task_type} - {res.status_code}")
                    else:
                      print(f"❌ Submit failedd: {task_type} - {res.status_code}")
                except Exception as e:
                    print(f"Submit exception: {e}")
        else:

            print(f"Submit Failure {task_type}")
            res = self._api_post(f"/submit/{task_id}", {"status": "fail", "client_id": CLIENT_ID})



    async def run(self):


        try:
            while True:
                task = self.get_task()
                if task:
                    now = time.time()
                    if now - self.last_heartbeat > HEARTBEAT_INTERVAL:
                        self.send_heartbeat(task.get("task_id"))
                        self.last_heartbeat = now
                    await self.handle_task(task)
                else:
                    print("⏳ No task available, sleeping...")
                    await asyncio.sleep(10)
        finally:
            print("main error")
if __name__ == "__main__":
    import sys
    import asyncio
    import nest_asyncio

    nest_asyncio.apply()

    ollama = OllamaRunner()
    smartypants = Smartypants(ollama)
    lookup_engine = LookupEngine( ollama)  # Proper initialization

    async def main():

        await lookup_engine.crawler_manager.start()
        client = NearestDoorClient(smartypants, lookup_engine, ollama)
        await client.run()

    try:

        asyncio.run(main())
    except KeyboardInterrupt:
        print("\n🛑 Shutting down gracefully...")
        sys.exit(0)

nohup: appending output to 'nohup.out'


[INIT].... → Crawl4AI 0.6.3 

📡 GET → /next-task
📡 POST → /heartbeat
🫀 Heartbeat sent.
▶️ Handling task search (ID: 30885)
{'task_id': 30885, 'task_type': 'search', 'object_type': 'shop', 'data': {}, 'target': {'id': 31799, 'name': 'Doug phillips dog park', 'city': 'Fishkill', 'state': 'New York', 'website': None, 'slug': 'doug-phillips-dog-park', 'shop_type': 'dog-park'}}
🌐 Starting combined search…
🔎 Google search → Doug phillips dog park Fishkill New York dog-park 
dd
ddf
bfd
ddd
 url canidate
https://www.bringfido.com/attraction/2330 url canidate
Fetching: https://www.bringfido.com/attraction/2330
No meta description found for https://www.bringfido.com/attraction/2330
ddj
❌ Extracted content too short: 23 chars
no snippet
https://www.fishkillrecreation.com/town-of-fishkill-parks.html url canidate
Fetching: https://www.fishkillrecreation.com/town-of-fishkill-parks.html
ddj
🧠 Running Ollama: qwen3:0.6b
false
Is the website https://www.fishkillrecreation.com/town-of-fishkill-parks.html related to 'Doug phillips do

[FETCH]... ↓ https://www.mapquest.com/us/new-york/doug-phillips-dog-park-305822738                                |
✓ | ⏱: 2.94s 

[SCRAPE].. ◆ https://www.mapquest.com/us/new-york/doug-phillips-dog-park-305822738                                |
✓ | ⏱: 0.00s 

...

[COMPLETE] ● https://www.mapquest.com/us/new-york/doug-phillips-dog-park-305822738                                |
✓ | ⏱: 15.16s 

CrawlResultContainer([CrawlResult(url='https://www.mapquest.com/us/new-york/doug-phillips-dog-park-305822738', html='<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n<title>ERROR: The request could not be satisfied</title>\n</head><body>\n<h1>403 ERROR</h1>\n<h2>The request could not be satisfied.</h2>\n<hr noshade="" size="1px">\nThe Amazon CloudFront distribution is configured to block access from your country.\nWe can\'t connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.\n<br clear="all">\nIf you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.\n<br clear="all">\n<hr noshade="" size="1px">\n<pre>Generated by cloudfront (Clo

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



main error

🛑 Shutting down gracefully...
Traceback (most recent call last):
  File "<ipython-input-59-b47b87188cdd>", line 189, in _extract_snippet
    desc = await page.locator('meta[name="description"]').get_attribute('content')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/playwright/async_api/_generated.py", line 16738, in get_attribute
    await self._impl_obj.get_attribute(name=name, timeout=timeout)
  File "/usr/local/lib/python3.11/dist-packages/playwright/_impl/_locator.py", line 418, in get_attribute
    return await self._frame.get_attribute(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/playwright/_impl/_frame.py", line 628, in get_attribute
    return await self._channel.send("getAttribute", locals_to_params(locals()))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-package

TypeError: object of type 'NoneType' has no len()